# Student information
Student: Balthazar Neveu

🔗 [Github](https://github.com/balthazarneveu/MVA24-delire) | [Online page for this class lab session](https://balthazarneveu.github.io/MVA24-delire)

⭐ [Online HTML version of this notebook](https://balthazarneveu.github.io/MVA24-delire/TP_5/tp_5.html)

# TP Coding a GAN in Pytorch

Author : Alasdair Newson

alasdair.newson@telecom-paris.fr

## Objective:

The goal of this TP is to explore GANs applied to the mnist (and possibly cifar10) datasets.

We will start with the mnist dataset.

### Your task:
You need to add the missing parts in the code (parts between # --- START CODE HERE and # --- END CODE HERE or # FILL IN CODE)

First of all, let's load some packages

In [15]:

import matplotlib.pyplot as plt
import sys
import numpy as np
import pickle
import copy
import os
import matplotlib.pyplot as plt

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from torch.autograd import Variable
from torchvision.utils import save_image


def pytorch_to_numpy(x):
    return x.detach().numpy()


device = "cuda" if torch.cuda.is_available() else "cpu"
device

'cuda'

## 1/ Loading the data

We define a function to load the mnist or cifar10 datasets. Note, we normalise the data between -1 and 1 here (this is often the case for GANs).

In [18]:

# create a random seed for reproducible results
random_seed = 1
torch.manual_seed(random_seed)

transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,)),
])

# MNIST Dataset
mnist_trainset = datasets.MNIST(root='./mnist_data/', train=True, transform=transform, download=True)
mnist_testset = datasets.MNIST(root='./mnist_data/', train=False, transform=transform, download=False)

# create data loader with smaller dataset size
max_mnist_size = 1000
mnist_trainset_reduced = torch.utils.data.random_split(
    mnist_trainset, [max_mnist_size, len(mnist_trainset)-max_mnist_size])[0]
mnist_train_loader = torch.utils.data.DataLoader(mnist_trainset_reduced, batch_size=64, shuffle=True)

# download test dataset
max_mnist_size = 512
mnist_testset_reduced = torch.utils.data.random_split(
    mnist_testset, [max_mnist_size, len(mnist_testset)-max_mnist_size])[0]
mnist_test_loader = torch.utils.data.DataLoader(mnist_testset_reduced, batch_size=64, shuffle=True)

In [19]:
N_ROWS = mnist_trainset_reduced.dataset.train_data.shape[1]
N_COLS = mnist_trainset_reduced.dataset.train_data.shape[2]
N_CHANNELS = 1
N_PIXELS = N_ROWS*N_COLS

img_shape = (N_ROWS, N_COLS, N_CHANNELS)

/home/bneveu/.local/lib/python3.10/site-packages/torchvision/datasets/mnist.py:75: UserWarning: train_data has been renamed data
  warnings.warn("train_data has been renamed data")


## 2/ GAN parameters

In [20]:

# GAN parameters
Z_DIM = 10
BATCH_SIZE = 64
# parameters for training
n_epochs = 400
n_iters_inner = 1  # number of internal loops
sample_interval = 100
# Learning rate for optimizers
lr = 0.0002
beta_1 = 0.5

# hidden dimensions : careful, the order here is with respect to the generator, and the discriminator is in the opposite order
H_DIM_1 = 256
H_DIM_2 = 512

## 3/ Model architecture

Now, we define the model architecture.

For the first dataset, mnist, we are going to use fully connected layers. Implement the following architecture, for the generator and the discriminator :

Generator :
- Dense layer, to size 256
- Leaky ReLU ($\alpha=0.2$)
- Dense layer, to size 512
- Leaky ReLU ($\alpha=0.2$)
- Dense layer, output size 784
- Tanh activation
- Reshape to size $28 \times 28 \times 1$

In [21]:
# Generator Code
class AdversarialElement(nn.Module):
    def __init__(
        self,
        z_dim: int = Z_DIM,
        h_dim_1: int = H_DIM_1,
        h_dim_2: int = H_DIM_2,
        n_rows: int = N_ROWS,
        n_cols: int = N_COLS,
        n_channels: int = N_CHANNELS
    ):
        super(AdversarialElement, self).__init__()
        self.n_rows = n_rows
        self.n_cols = n_cols
        self.n_channels = n_channels
        self.n_pixels = (self.n_rows)*(self.n_cols)
        self.h_dim_1 = h_dim_1
        self.h_dim_2 = h_dim_2
        self.z_dim = z_dim
        self.n_pixels = (self.n_rows)*(self.n_cols)
        self.non_linear = torch.nn.LeakyReLU(0.2)

    def forward(self, x):
        raise NotImplementedError


class Generator(AdversarialElement):
    def __init__(self, **kwargs):
        super().__init__(**kwargs)
        self.normalize_values = torch.nn.Tanh()
        self.fc1 = torch.nn.Linear(self.z_dim, self.h_dim_1)
        self.fc2 = torch.nn.Linear(self.h_dim_1, self.h_dim_2)
        self.fc3 = torch.nn.Linear(self.h_dim_2, self.n_pixels*self.n_channels)

        self.fwd = torch.nn.Sequential(
            self.fc1,
            self.non_linear,
            self.fc2,
            self.non_linear,
            self.fc3,
            self.normalize_values
        )

    def forward(self, z: torch.Tensor) -> torch.Tensor:
        """Convert noise vector z into an image-like tensor.

        Args:
            z (torch.Tensor): [N, Z] Noise tensor of size (batch_size, z_dim)

        Returns:
            x (torch.Tensor): [N, C, H, W] Image tensor of size (batch_size, n_channels, n_rows, n_cols)
        """
        x = self.fwd(z)
        x = x.view(-1, self.n_channels, self.n_rows, self.n_cols)
        return x

Discriminator :
- Flatten
- Dense layer, to size 512
- Leaky ReLU ($\alpha=0.2$)
- Dense layer, output size 256
- Leaky ReLU ($\alpha=0.2$)
- Dense layer, output size 1
- Sigmoid activation

In [23]:
class Discriminator(AdversarialElement):
    def __init__(self, **kwargs):
        super().__init__(**kwargs)
        self.fc1 = nn.Linear(self.n_pixels*self.n_channels, self.h_dim_2)
        self.fc2 = nn.Linear(self.h_dim_2, self.h_dim_1)
        self.fc3 = nn.Linear(self.h_dim_1, 1)
        self.normalize_values = torch.nn.Sigmoid()
        self.fwd = torch.nn.Sequential(
            self.fc1,
            self.non_linear,
            self.fc2,
            self.non_linear,
            self.fc3,
            self.normalize_values
        )

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        """Discriminator forward function.

        Args:
            x (torch.Tensor): [N, C, H, W] Image tensor of size (batch_size, n_channels, n_rows, n_cols)

        Returns:
            torch.Tensor: probability of the image being real (after sigmoid)
        """
        y = x.view(-1, self.n_pixels*self.n_channels)
        y = self.fwd(y)
        return y

gen_test = Generator(n_channels=4).to(device)
out_gen = gen_test(torch.randn(BATCH_SIZE, gen_test.z_dim).to(device))
assert out_gen.shape == (
    BATCH_SIZE, gen_test.n_channels, gen_test.n_rows, gen_test.n_cols), "Wrong output size"

disc_test = Discriminator(n_channels=gen_test.n_channels).to(device)
proba_real = disc_test(out_gen)
assert proba_real.shape == (
    BATCH_SIZE, 1), "Wrong output size"
del gen_test, disc_test, out_gen, proba_real

Create generator and discriminator 

In [25]:
gen_model = Generator().to(device)
# Print the model
print(gen_model)

disc_model = Discriminator() # FILL IN HERE
# Print the model
print(disc_model)

Generator(
  (non_linear): LeakyReLU(negative_slope=0.2)
  (normalize_values): Tanh()
  (fc1): Linear(in_features=10, out_features=256, bias=True)
  (fc2): Linear(in_features=256, out_features=512, bias=True)
  (fc3): Linear(in_features=512, out_features=784, bias=True)
  (fwd): Sequential(
    (0): Linear(in_features=10, out_features=256, bias=True)
    (1): LeakyReLU(negative_slope=0.2)
    (2): Linear(in_features=256, out_features=512, bias=True)
    (3): LeakyReLU(negative_slope=0.2)
    (4): Linear(in_features=512, out_features=784, bias=True)
    (5): Tanh()
  )
)
Discriminator(
  (non_linear): LeakyReLU(negative_slope=0.2)
  (fc1): Linear(in_features=784, out_features=512, bias=True)
  (fc2): Linear(in_features=512, out_features=256, bias=True)
  (fc3): Linear(in_features=256, out_features=1, bias=True)
  (normalize_values): Sigmoid()
  (fwd): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): LeakyReLU(negative_slope=0.2)
    (2): Linear(in_featu

## 4/ Loss function


The GAN loss function is the following :
\begin{equation}
	\min_{G} \max_{D} \mathbb{E}_{x \in p_{data}} \left[ \log D(x)\right] +
	\mathbb{E}_{z \in p_{z}}\left[ \log \left( 1 - D(G(z)) \right)\right],
\end{equation}
where $G$ is the generator, $D$ is the discriminator, $z$ is the latent code, which follows a normal distribution.

You should notice that this is extremely similar to the binary cross-entropy function. Therefore, there is an intelligent way to train the discriminator : we give it first a batch of real images, and label them as real, and secondly we give a batch of fake images and label them as fake. Therefore, the discriminator training itself is done in two sequential steps (first true, then fake). If the labels are correctly chosen (further on, during training), you can (and __should__) use the binary cross-entropy function.

The generator loss, however, must be specified as :
- $mean(\log(1-D(G(z))))$

You can use the ```torch.mean``` function for this purpose.


The training is carried out sequentially : first we execute a few training steps on the discriminator, and then one on the generator. Therefore, we use two loops : one to train the discriminator (the internal loop) and one to train the generator (external loop, ie. the number of epochs). The GAN training algorithm is as follows :

- For $i=0$ to $n-1$
  - For $j=0$ to $m-1$
    - $x \leftarrow$ random batch of data
    - $z \leftarrow$ random batch of latent codes
    - Train discriminator on real images $x$
    - Train discriminator on fake images $G(z)$
  - $z \leftarrow$ random batch of latent codes
  - Train discriminator on fake images $G(z)$



In [ ]:

# Setup Adam optimizers for both G and D
optimizer_disc = optim.Adam(disc_model.parameters(), lr=lr, betas=(beta_1, 0.999))
optimizer_gen = optim.Adam(gen_model.parameters(), lr=lr, betas=(beta_1, 0.999))

# criterion used for the discriminator loss
bce_criterion = ...  # FILL IN HERE

# criterion used for the generator loss


def loss_fn_gen(d_gen_data):
    loss_gen = ...  # FILL IN CODE
    return loss_gen

### Sampling function

We now create a function to sample several images during training (to follow the convergence of the network)

In [ ]:

def sample_images(generator, z_dim, rand_seed=30):
    # np.random.seed(rand_seed)
    r, c = 5, 5
    z_random = torch.randn(r * c, 1, z_dim, dtype=torch.float, device=device)  # np.random.normal(0, 1, (r * c, z_dim))

    gen_imgs = np.transpose(generator(z_random).cpu().detach().numpy(), (0, 2, 3, 1))

    # Rescale images 0 - 1
    gen_imgs = 0.5 * gen_imgs + 0.5

    fig, axs = plt.subplots(r, c)
    cnt = 0
    for i in range(r):
        for j in range(c):
            # black and white images
            if (gen_imgs.shape[3] == 1):
                axs[i, j].imshow(gen_imgs[cnt, :, :, 0], cmap='gray')
            elif (gen_imgs.shape[3] == 3):  # colour images
                gen_imgs_temp = gen_imgs.copy()
                gen_imgs_temp = 255.*np.clip(gen_imgs_temp, 0.0, 1.0)
                axs[i, j].imshow(gen_imgs_temp[cnt, :, :, :].astype(np.uint8))
            else:
                print('Error, unsupported channel size. Dude, I don''t know what you want me to do.\
            I can''t handle this data. You''ve made me very sad ...')
            axs[i, j].axis('off')
            cnt += 1
    plt.show()

## 5/ Training

We are now ready to train the network. Here is the training algorithm again :

- For $i=0$ to $n-1$
  - For $j=0$ to $m-1$
    - $x \leftarrow$ random batch of data
    - $z \leftarrow$ random batch of latent codes
    - Train discriminator on real images $x$
    - Train discriminator on fake images $G(z)$
  - $z \leftarrow$ random batch of latent codes
  - Train discriminator on fake images $G(z)$

You can use ```np.random.normal``` to create a batch of random latent codes, and ```np.random.randint``` to create a batch of random images.

You can then train the discriminator and the generator using the ```train_on_batch``` function.

We do not worry here about looping over the whole database : just create a random batch at each iteration.

In [ ]:

# Training Loop

# Lists to keep track of progress
img_list = []
G_losses = []
D_losses = []

sample_interval = 25

print("Starting Training")
# For each epoch
for epoch in range(n_epochs):
    # For each batch in the dataloader
    for i, data in enumerate(mnist_train_loader, 0):
        for iter_inner in range(0, n_iters_inner):

            ############################
            # Train discriminator
            ############################
            # Train with true data batch
            disc_model.zero_grad()
            # create true data and labels
            true_imgs = ...  # FILL IN HERE
            true_labels = ...  # FILL IN HERE
            # put true data through discriminator
            d_output_true = ...  # FILL IN HERE
            # bce loss on true data
            d_loss_true = ...  # FILL IN HERE
            # backpropagation for discriminator, true loss
            d_loss_true.backward()
            disc_true_value = d_output_true.mean().item()

            # Train with fake data batch
            # create fake data and labels
            # generate batch of random latent vectors
            z_latent_noise = ...  # FILL IN HERE
            # Generate batch of fake images
            fake_imgs = ...  # FILL IN HERE
            fake_labels = ...  # FILL IN HERE
            # put fake data through discriminator
            disc_output_fake = ...  # FILL IN HERE
            # bce loss on fake data
            disc_loss_fake = ...  # FILL IN HERE
            # backpropagation for discriminator, fake loss
            disc_loss_fake.backward()
            disc_fake_value = disc_output_fake.mean().item()
            # Update discriminator
            optimizer_disc.step()

            d_loss_total = d_loss_true+disc_loss_fake

        ############################
        # Train generator
        ############################
        gen_model.zero_grad()
        # We have updated the discriminator, so we need to update the output of the discriminator
        disc_gen_output_fake = ...  # FILL IN HERE
        # Generator loss, using the custom loss
        g_loss = ...  # FILL IN HERE
        # backpropagation for generator
        g_loss.backward()
        # D_G_z2 = output.mean().item()
        # Update generator
        optimizer_gen.step()

        # Output training stats
        if i % 200 == 0:
            print('[%d/%d][%d/%d] \tLoss_D: %.4f\tLoss_G: %.4f\tD(x): %.4f\tD(G(z)): %.4f'
                  % (epoch, n_epochs, i, len(mnist_train_loader), d_loss_total.item(), g_loss.item(), disc_true_value, disc_fake_value))

        # Save Losses for plotting later
        G_losses.append(g_loss.item())
        D_losses.append(d_loss_total.item())

    if (epoch % sample_interval == 0):
        sample_images(gen_model, z_dim, rand_seed=30)

# end samples
sample_images(gen_model, z_dim, rand_seed=30)

## 6/ Navigating in the latent space

One of the main interests in the latent space is to be able to __navigate__ in it. For instance, one operation which is very common is to take two points $z_0$ and $z_1$ and to interpolate between the two. The images resulting from the generation of the interpolated points should ideally be a mix between the two initial points. 

The simplest method of interpolation is simply linear interpolation along the line connecting the two points. Obviously, this supposes that the latent space is linear in some sense, which may not be true, but we are going to take this approach here. 

First, find two inital points $z_0$ and $z_1$ which you feel represent two different digits well, and display their resulting generated images.  

In [ ]:
# first image
random_seed = 110  # change this seed to find the best image
torch.manual_seed(random_seed)
z_0 = ...  # FILL IN HERE
x_0 = ...  # FILL IN HERE

random_seed = 64  # change this seed to find the best image
torch.manual_seed(random_seed)
z_1 = ...  # FILL IN HERE
x_1 = ...  # FILL IN HERE

# display images
# FILL IN HERE

Now, carry out the interpolation between these two points. You should __include__ the starting and ending codes $z_0$ and $z_1$. Display the results.

In [ ]:
n_interpolation = 20

# FILL IN HERE

In [ ]:
# display results

fig, axs = plt.subplots(1, n_interpolation, figsize=(20, 20))
for i in range(n_interpolation):
    # black and white images
    axs[i].imshow(..., cmap='gray')  # FILL IN HERE
    axs[i].axis('off')
plt.show()

What do you think of the quality of the output images ?

__Answer__

## 7/ Evaluation of navigation

Now, we are going to evaluate this navigation. We will investigate two properties:

- 1/ We are going to see whether by navigating between two points, the model produces images which are categorised in either of the classes (beginning class or end class), or if it traverses a region which is categorised in another class;
- 2/ We are going to see whether the confidence of a classification network in different regions varies much. In other words, when we navigate in the latent space, do we go into zones which do not correspond to any number (according to the classification network) ? 

For this, we will first need a classification network. Take the code from the last lab work on variational autoencoders, and extract the section where we trained a classifier on mnist. Here is the architecture we used:

- conv2d, filter size  3×3 , 32 filters, stride=(2,2), padding="SAME"
- ReLU
- conv2d, filter size  3×3 , 32 filters, stride=(2,2), padding="SAME"
- ReLU
- MaxPool2D, stride=(2,2)
- Flatten
- Dense layer

In [ ]:
# define mnist classifier model, loss function, optimiser and the function 'get_accuracy'

# FILL IN HERE

In [ ]:
# training the classifier

# FILL IN HERE

### 7.1/ Evaluation of navigation 1

Now, create a function which returns the classification returned by this network on mnist images. Remember, the classification model above returns the values __before__ the softmax is applied, and here we want the argmax, rather than the maximum probability. __Note__: the torch.max function returns a tuple: (max_values, max_indices).


In [ ]:
def predict_mnist_class(imgs_in, classification_model):
    output_classes = ...  # FILL IN HERE
    return (output_classes)

Carry out and print the classification of your interpolated latent space images. What are your remarks ? Does the latent space navigation traverse regions with classes other than those of $z_0$ and $z_1$ ?

In [ ]:
z_classes = ...  # FILL IN HERE
print(z_classes)

__Answer__

### 7.2/ Evaluation of navigation 2

Even though the network may predict coherent image classes during the navigation, it may predict these classes with different confidences. Ideally, we would like the latent space to contain codes which produce images of constant quality. Therefore, as mentioned above, we want to check these prediction confidences during navigation on our GAN.

Carry this out now. First of all, define a function which, instead of funding the output *classes* of the interpolations, find the *probability* of the most likely class (the same operation as in the lab on variational autoencoders).


In [ ]:
def predict_mnist_confidence(imgs_in, classification_model):
    output_classes = ...  # FILL IN HERE
    return (output_classes)

Using this function, carry out the second evaluation on the images generated by interpolated latent codes.

In [ ]:
z_confidence = ...  # FILL IN HERE
print(z_confidence)

Does the confidence change greatly during interpolation ? Does the navigation go through zones where the classification is not confident ? Why do you think this is ? (think about the form/shape of the latent space).

__Answer__

# __Optional__ : Training on CIFAR

If you want to try another, more challenging database, use the above code and modify it to carry out the GAN training on the CIFAR10 database. Note, it can take a long time to get good results

First, we download the data.

In [ ]:
# convert input to Pytorch tensors
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,)),
])

# extract mnist data
cifar_trainset = datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
print(cifar_trainset)

# create data loader with smaller dataset size
max_cifar_size = 2000
cifar_trainset_reduced = torch.utils.data.random_split(
    cifar_trainset, [max_cifar_size, len(cifar_trainset)-max_cifar_size])[0]
cifar_train_loader = torch.utils.data.DataLoader(cifar_trainset_reduced, BATCH_SIZE=64, shuffle=True)

# download test dataset
cifar_testset = datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)
cifar_test_loader = torch.utils.data.DataLoader(cifar_testset, BATCH_SIZE=64, shuffle=True)

n_rows = 32
n_cols = 32
n_channels = 3
n_pixels = n_rows*n_cols

img_shape = (n_rows, n_cols, n_channels)

Now, we can redefine the hyper-parameters of the model (change if you wish)

In [ ]:
# GAN parameters
z_dim = 10
BATCH_SIZE = 64
n_epochs = 300
# parameters for training
n_iters_inner = 1  # number of internal loops
# Learning rate for optimizers
lr = 0.0002
beta_1 = 0.5

For this case of CIFAR, implement the following architecture :

- Generator :
  - Dense layer to size 1024
  - Leaky ReLU ($\alpha=0.2$)
  - Reshape, to size $4 \times 4 \times64$
  - % size = $4\times4\times64$
  - Conv2d, n_channels=16,kernel size=(3,3), strides=(1,1),padding=(1,1)
  - Upsample(scale_factor=(2,2))
  - %size = $8\times 8\times 16$
  - Leaky ReLU ($\alpha=0.2$)
  - Conv2d, n_channels=16,kernel size=(3,3), strides=(1,1),padding=(1,1)
  - Upsample(scale_factor=(2,2))
  - %size=$16 \times 16 \times 16$
  - Leaky ReLU ($\alpha=0.2$)
  - Conv2d, n_channels=3,kernel size=(3,3), strides=(1,1),padding=(1,1)
  - Upsample(scale_factor=(2,2))
  - %size = $32 \times 32 \times 3$
  - Tanh activation ( you can use ```Activation('tanh')```)

- Discriminator :
  - % input size : $32 \times 32 \times 3$
  - Conv2D, 32 filters, kernel size = (3,3), strides = (1,1),padding = same
  - % size $32 \times 32 \times 32$
  - Leaky ReLU ($\alpha=0.2$)
  - Conv2D, 32 filters, kernel size = (3,3), strides = (2,2),padding = same
  - %size : $16 \times 16 \times 32$
  - Leaky ReLU ($\alpha=0.2$)
  - Conv2D, 64 filters, kernel size = (3,3), strides = (2,2),padding = same
  - % size : $8 \times 8 \times 64$
  - Leaky ReLU ($\alpha=0.2$)
  - Conv2D, 32 filters, kernel size = (3,3), strides = (2,2),padding = same
  - % size : $4 \times 4 \times 32$
  - Leaky ReLU ($\alpha=0.2$)
  - Flatten
  - Dense layer to size 1
  - Sigmoid activation

  Implement this architecture below, and train the GAN. 

In [ ]:
# GAN implementation

# FILL IN HERE

Now, carry out the training (use code above)

In [ ]:
# training code

# FILL IN HERE